<a href="https://colab.research.google.com/github/SnehaKellogg/wrds_pthon/blob/master/MutualFund_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import dependencies
# !pip install wrds # required only first time when this file is run on a system

import wrds
import pandas as pd
import sqlalchemy 
import psycopg2 

     |████████████████████████████████| 2.9MB 4.1MB/s 


## Setting up connection

In [0]:
# enter your username password here:

conn = psycopg2.connect(dbname='wrds', 
                        user='', 
                        host='wrds-pgdata.wharton.upenn.edu', 
                        port=9737,
                        password='')

### Optional 

In [0]:
#run this only if table decription etc is required -- we are not using this otherwise
# db = wrds.Connection(wrds_username='dsk3316')
# db.describe_table('crspq','daily_nav_ret')

List of db funtionc available:
db.close()
db.connection()
db.describe_table()
db.get_table()
db.list_tables()
db.raw_sql()
db.get_row_count()
db.list_libraries()

help(db.get_table)
help(db.raw_sql)

db.get_table('djones', 'djdaily', columns=['date', 'dji'], obs=10)

db.raw_sql('select date,dji from djones.djdaily LIMIT 10;', date_cols=['date'])

In [0]:
# define file upload , run the next cell when this is 100% complete
from google.colab import files
uploaded = files.upload()

Saving tickers.csv to tickers.csv


In [0]:
import io
inputfile = pd.read_csv(io.BytesIO(uploaded['tickers.csv']))

In [0]:
#upload the tickermapping csv here -- always make sure the filename is exactly same here and in next cell
uploaded = files.upload()

Saving tickermapping.csv to tickermapping.csv


In [0]:
#Creating a dataframe using the uploaded file
mappingtable = pd.read_csv(io.BytesIO(uploaded["tickermapping.csv"]))

In [0]:
# Yet to check whether the same ticker / fund no is used in the history which might creates errors
# For now, Keeping only the desired columns and Removing multiples to create unique matches 
mappingtable = mappingtable.drop(['summary_period2', 'caldt', 'fund_name'], axis=1)
mappingtable = mappingtable.dropna() #there were NaN values in ticker for many of them
mappingtable = mappingtable.drop_duplicates(subset=['ticker'],keep = 'last')
mappingtable = mappingtable.reset_index(drop= True)
mappingtable.sort_values(by='ticker',inplace=True)

In [0]:
# Merging based on the tickers- one to one didnt happen until I dropped all duplicates of ticker 
## -- needs to checked-- currently it keeps the last observation sorted on date
required_mfs = pd.merge(left = inputfile, right = mappingtable, how = 'left',on = 'ticker', validate = "one_to_one")

In [0]:
# Check if all tickers were found
print(inputfile.shape)
print(required_mfs.shape)
# this matches the input file shape so all the tickers were found.

(91, 1)
(91, 2)


In [0]:
#Create a list of crsp_fundno to be used
fund_number=list(required_mfs['crsp_fundno'].values)
len(fund_number)

91

In [0]:
# # Checking connection and statement with single query, keeping this if required to run
# mfdaily = pd.read_sql_query("""SELECT * FROM crspq.daily_nav_ret where crsp_fundno =3173;""", conn)

# # mfdaily = db.raw_sql("SELECT * FROM crspq.daily_nav_ret where crsp_fundno =3173")
# mfdaily.head()

#### pull in daily return data for 10 years of each mutual fund and populate one database

In [0]:
#Create an empty dataframe to add all values 
dailyreturns = pd.DataFrame()

for fundno in fund_number:
    query = f'SELECT * FROM crspq.daily_nav_ret WHERE crsp_fundno = {fundno};'
    mfdaily =  pd.read_sql_query(query,conn)
    dailyreturns = dailyreturns.append(mfdaily)
#     print(mfdaily.shape) #just to check if it's returning something
    

dailyreturns.shape

In [0]:
dailyreturns.head()

,crsp_fundno,caldt,dnav,dret
0,3173.0,2004-04-30,10.03,NaN
1,3173.0,2004-05-03,10.02,-0.000997
2,3173.0,2004-05-04,10.02,0.000000
3,3173.0,2004-05-05,10.01,-0.000998
4,3173.0,2004-05-06,10.01,0.000000


### Exporting everything to excel

In [0]:
# Save files to single file
!pip install xlsxwriter #required to run only first time
import xlsxwriter

writer = pd.ExcelWriter('mfreturns.xlsx', engine='xlsxwriter')

dailyreturns.to_excel(writer, sheet_name='returns')
required_mfs.to_excel(writer, sheet_name='fundno_ticker')


writer.save()

     |████████████████████████████████| 143kB 2.8MB/s 


In [0]:
#for colab only

files.download('mfreturns.xlsx')

In [0]:
# CREATE CSV SO I CAN UPLOAD INTO STATA
# FOR HERE, ONE REGRESSION TO START ME OFF PLEASE:  regress dailyreturn l.dailyreturn l2.dailyreturn
# FOR HERE, ONE REGRESSION TO START ME OFF PLEASE:  regress dailyreturn dummy=1ifpriorTHREEdaysdailyreturn=0 dummy=1ifpriorFOURdaysdailyreturn=0 FIVE SIX
